In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, random_split
import random
import torch.nn as nn
        
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA on GPU")
else:
    device = torch.device("cpu")
    print("CUDA not availible, using CPU")

# Завантаження даних
data_path = 'IMDB_Sentiment/train.csv'
data = pd.read_csv(data_path)

# Перевірка розміру завантаженого датасету
print(f"Загальна кількість рядків: {len(data)}")

# Вибір негативних і позитивних відгуків
positive_reviews = data[:12500]
negative_reviews = data[12500:25000]

# Перевірка розмірів вибірок
print(f"Кількість негативних відгуків: {len(negative_reviews)}")
print(f"Кількість позитивних відгуків: {len(positive_reviews)}")

# Фіксація генератора випадкових чисел для повторюваності
random.seed(42)
test_negative = negative_reviews.sample(2500)
test_positive = positive_reviews.sample(2500)

# Перевірка коректного видалення вибірки з оригіналу
train_negative = negative_reviews.drop(test_negative.index)
train_positive = positive_reviews.drop(test_positive.index)

# Об'єднання тренувальних та тестових даних
train_data = pd.concat([train_negative, train_positive]).reset_index(drop=True)
test_data = pd.concat([test_negative, test_positive]).reset_index(drop=True)

Using CUDA on GPU
Загальна кількість рядків: 25000
Кількість негативних відгуків: 12500
Кількість позитивних відгуків: 12500


In [2]:
from transformers import BertTokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the input texts
def tokenize(data):
    return tokenizer(
        data.tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

train_encodings = tokenize(train_data['text'])
test_encodings = tokenize(test_data['text'])

# Convert labels to tensors

train_labels = torch.tensor(train_data['sentiment'].values)
test_labels = torch.tensor(test_data['sentiment'].values)

In [3]:
from torch.utils.data import Dataset
from transformers import BertForSequenceClassification

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)


#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
from transformers import Trainer, TrainingArguments
import evaluate
#from transformers import TrainerCallback
#import matplotlib.pyplot as plt
#
#class AccuracyPlotCallback(TrainerCallback):
#    def __init__(self):
#        super().__init__()
#        self.epoch_accuracies = []
#
#    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
#        # Append the evaluation accuracy to the list at each epoch
#        if metrics and "eval_accuracy" in metrics:
#            self.epoch_accuracies.append(metrics["eval_accuracy"])
#
#    def plot(self):
#        # Plot the accuracy over epochs
#        plt.plot(self.epoch_accuracies, label="Validation Accuracy")
#        plt.xlabel("Epoch")
#        plt.ylabel("Accuracy")
#        plt.title("Model Accuracy over Epochs")
#        plt.legend()
#        plt.show()
#
#
#accuracy_callback = AccuracyPlotCallback()
# Load accuracy metric
accuracy_metric = evaluate.load("accuracy")

# Define a compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=5,              
    per_device_train_batch_size=128,  
    per_device_eval_batch_size=128,  
    save_strategy="epoch",
    warmup_steps=100,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,                        
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics#,
    #callbacks=[accuracy_callback]
)

D:\Programming\Anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
trainer.train()
accuracy_callback.plot()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.020200,0.610549,0.875200
2,0.022600,0.742727,0.875800


KeyboardInterrupt: 

In [52]:
results = trainer.evaluate()
print("Test Accuracy:", results)

Test Accuracy: {'eval_loss': 0.6007522344589233, 'eval_model_preparation_time': 0.002, 'eval_accuracy': 0.883, 'eval_runtime': 16.1398, 'eval_samples_per_second': 309.793, 'eval_steps_per_second': 2.478}


In [38]:
model.save_pretrained('./sentiment-model-2')
tokenizer.save_pretrained('./sentiment-model-2')

('./sentiment-model-2\\tokenizer_config.json',
 './sentiment-model-2\\special_tokens_map.json',
 './sentiment-model-2\\vocab.txt',
 './sentiment-model-2\\added_tokens.json')

In [42]:
from transformers import AutoModelForSequenceClassification

# Load the saved model
model = AutoModelForSequenceClassification.from_pretrained('./sentiment-model')

from transformers import AutoTokenizer

# Load the saved tokenizer
tokenizer = AutoTokenizer.from_pretrained('./sentiment-model')

In [44]:
results = model.evaluate()
print("Test Accuracy:", results)

AttributeError: 'BertForSequenceClassification' object has no attribute 'evaluate'